In [1]:
import pandas as pd
from pathlib import Path 

#create path for both CSV data sets
census_data_path = r"Resources\clean_census_data.csv"
cw_data_path = r"Resources\CWData.csv"

# Store the paths and command to read
census_data = pd.read_csv(census_data_path)
cw_data = pd.read_csv(cw_data_path)





In [2]:
# Turn Row 1 values into Column Headers & delete Row 1 containing prev. Column Headers
census_data.rename(columns=census_data.iloc[0], inplace = True)
census_data.drop(census_data.index[0], inplace=True)
census_data.head()

,County,Total,"Less than $10,000","$10,000 to $14,999","$15,000 to $24,999","$25,000 to $34,999","$35,000 to $49,999","$50,000 to $74,999","$75,000 to $99,999","$100,000 to $149,999","$150,000 to $199,999","$200,000 or more",Median income (dollars),Mean income (dollars),Household income in the past 12 months,Family income in the past 12 months,Nonfamily income in the past 12 months
1,Angelina,"34,276",5.1%,3.9%,10.5%,9.5%,15.6%,18.0%,11.3%,12.9%,8.4%,4.7%,"58,180","79,881",39.7%,(X),(X)
2,Bastrop,"37,278",6.3%,3.6%,6.1%,5.3%,12.6%,19.1%,11.8%,21.5%,8.9%,4.9%,"72,663","87,467",41.1%,(X),(X)
3,Bell,"143,361",6.7%,3.6%,6.9%,8.5%,14.1%,20.2%,13.7%,14.7%,5.5%,6.1%,"62,137","85,250",35.1%,(X),(X)
4,Bexar,"754,632",5.3%,4.4%,7.9%,8.0%,11.9%,18.6%,13.7%,14.9%,7.9%,7.4%,"65,854","89,010",33.2%,(X),(X)
5,Bowie,"35,469",8.7%,5.9%,10.3%,9.9%,15.3%,20.3%,9.3%,9.8%,4.6%,5.8%,"49,771","74,515",42.2%,(X),(X)


In [3]:
# Convert Allegation Types into Columns
pivoted_df = cw_data.pivot_table(index="County", columns="Type of Allegation", values="Allegation Count", aggfunc='first')
combined_cw = pd.merge(pivoted_df, cw_data, how = "right", on=["County", "County"])
combined_cw.head()

,County,Abandonment,Emotional Abuse,Labor Trafficking,Medical Neglect,Neglectful Supervision,Physical Abuse,Physical Neglect,Refusal To Accept Parental Responsibility,Sex Trafficking,Sexual Abuse,Fiscal Year,Region,Type of Allegation,Allegation Count
0,Armstrong,NaN,1.0,NaN,1.0,11.0,5.0,1.0,NaN,NaN,1.0,2022,1-Lubbock,Neglectful Supervision,11
1,Armstrong,NaN,1.0,NaN,1.0,11.0,5.0,1.0,NaN,NaN,1.0,2022,1-Lubbock,Physical Abuse,5
2,Armstrong,NaN,1.0,NaN,1.0,11.0,5.0,1.0,NaN,NaN,1.0,2022,1-Lubbock,Sexual Abuse,1
3,Bailey,NaN,1.0,NaN,2.0,50.0,19.0,9.0,1.0,NaN,15.0,2022,1-Lubbock,Emotional Abuse,1
4,Bailey,NaN,1.0,NaN,2.0,50.0,19.0,9.0,1.0,NaN,15.0,2022,1-Lubbock,Neglectful Supervision,50


In [4]:
# Drop duplicate rows ==> 1 per County
combined_cw = combined_cw.drop_duplicates(subset = "County", ignore_index = True)

# I goofed somewhere but keeping this in case I need it later
# combined_cw = combined_cw.drop(columns = ['Type of Allegation', 'Allegation Count', 'Neglectful Supervision_y', 
#                                           'Physical Abuse_y', 'Sexual Abuse_y', 'Emotional Abuse_y', 
#                                           'Physical Neglect_y', 'Medical Neglect_y', 
#                                           'Refusal To Accept Parental Responsibility_y',
#                                           'Labor Trafficking_y', 'AbandonmentSex Trafficking'])
# combined_cw = combined_cw.rename(columns = {'Emotional Abuse_x' : 'Emotional Abuse',
#                                             'Labor Trafficking_x': 'Labor Trafficking',
#                                             'Medical Neglect_x': 'Medical Neglect',
#                                             'Neglectful Supervision_x': 'Neglectful Supervision',
#                                             'Physical Abuse_x' : 'Physical Abuse',
#                                             'Physical Neglect_x' : 'Physical Neglect',
#                                             'Refusal To Accept Parental Responsibility_x' : 'Refusal To Accept Parental Responsibility',
#                                             'Sexual Abuse_x' : 'Sexual Abuse'})

# Turn NaN Cases into 0 Cases
combined_cw = combined_cw.fillna(0)

# Rename Total for clarity when merging
df = combined_cw.rename(columns={'Total':'Total Number of Households'})
combined_cw.head()

,County,Abandonment,Emotional Abuse,Labor Trafficking,Medical Neglect,Neglectful Supervision,Physical Abuse,Physical Neglect,Refusal To Accept Parental Responsibility,Sex Trafficking,Sexual Abuse,Fiscal Year,Region,Type of Allegation,Allegation Count
0,Armstrong,0.0,1.0,0.0,1.0,11.0,5.0,1.0,0.0,0.0,1.0,2022,1-Lubbock,Neglectful Supervision,11
1,Bailey,0.0,1.0,0.0,2.0,50.0,19.0,9.0,1.0,0.0,15.0,2022,1-Lubbock,Emotional Abuse,1
2,Briscoe,0.0,2.0,0.0,1.0,8.0,2.0,1.0,1.0,0.0,1.0,2022,1-Lubbock,Neglectful Supervision,8
3,Carson,2.0,1.0,0.0,4.0,31.0,12.0,3.0,1.0,1.0,12.0,2022,1-Lubbock,Emotional Abuse,1
4,Castro,0.0,5.0,1.0,8.0,45.0,19.0,6.0,0.0,1.0,13.0,2022,1-Lubbock,Emotional Abuse,5


In [14]:
# Merge both data sets into one   
happy_childhood_data = pd.merge(census_data, combined_cw, how="left", on=["County","County"])
happy_childhood_data.head()
happy_childhood_data.to_csv('Resources/full_clean_dataset.csv')


In [7]:
happy_childhood_data.count()

County                                        4435
Total                                         4435
    Less than $10,000                         4435
    $10,000 to $14,999                        4435
    $15,000 to $24,999                        4435
    $25,000 to $34,999                        4435
    $35,000 to $49,999                        4435
    $50,000 to $74,999                        4435
    $75,000 to $99,999                        4435
    $100,000 to $149,999                      4435
    $150,000 to $199,999                      4435
    $200,000 or more                          4435
Median income (dollars)                       4435
Mean income (dollars)                         4435
    Household income in the past 12 months    4435
    Family income in the past 12 months       4435
    Nonfamily income in the past 12 months    4435
Fiscal Year                                   4435
Region                                        4435
Type of Allegation             

In [8]:
#How many allegation types are there?
unique_allegations = happy_childhood_data['Type of Allegation'].nunique()
unique_allegations

10

In [9]:
#list allegation types
unique_allegations = happy_childhood_data['Type of Allegation'].unique()
unique_allegations

array(['Abandonment', 'Emotional Abuse', 'Medical Neglect',
       'Neglectful Supervision', 'Physical Abuse', 'Physical Neglect',
       'Refusal To Accept Parental Responsibility', 'Sex Trafficking',
       'Sexual Abuse', 'Labor Trafficking'], dtype=object)

In [10]:
#How many counties are there?
unique_counties = happy_childhood_data['County'].nunique()
unique_counties

51

In [11]:
#list county names
unique_counties = happy_childhood_data['County'].unique()
unique_counties

array(['Angelina', 'Bastrop', 'Bell', 'Bexar', 'Bowie', 'Brazoria',
       'Brazos', 'Cameron', 'Collin', 'Comal', 'Coryell', 'Dallas',
       'Denton', 'Ector', 'Ellis', 'Galveston', 'Grayson', 'Gregg',
       'Guadalupe', 'Harris', 'Harrison', 'Hays', 'Henderson', 'Hidalgo',
       'Hood', 'Hunt', 'Jefferson', 'Johnson', 'Kaufman', 'Liberty',
       'Lubbock', 'McLennan', 'Midland', 'Montgomery', 'Nueces', 'Orange',
       'Parker', 'Potter', 'Randall', 'Rockwall', 'Smith', 'Starr',
       'Tarrant', 'Taylor', 'Travis', 'Victoria', 'Walker', 'Webb',
       'Wichita', 'Williamson', 'Wise'], dtype=object)